# Automates
Voir par exemple les paragraphes 3.1, 5.1.1, 5.1.2, 5.1.3, 5.1.4, 5.2.1 et 5.2.2 de ce [poly](https://1drv.ms/b/s!Aqb1Bqn8IHzxmm0bQRQeJfZoOZux?e=XZf5aQ)
## 1. Déterminisation et recherche des mots de longueur minimale

In [1]:
from collections import deque

class AF:
    """Automate fini non deterministe."""
    
    def __init__(self, symboles, etats, etatsInitiaux, etatsFinaux, transitions):
        self.symboles = symboles
        self.etats = etats
        self.etatsInitiaux = etatsInitiaux
        self.etatsFinaux  = etatsFinaux
        self.transitions = transitions
        
    def delta0(self,q,x):
        """q est un etat, x une lettre"""
        try:
            return self.transitions[q][x]
        except KeyError :
            return set()

    def delta1(self,e,x):
        """e est un ensemble d'etats, x une lettre"""
        return set().union(*[self.delta0(q,x) for q in e])

    def afd_fonctionnel(self):
        return AFD_fonctionnel(
            symboles = self.symboles,
            etatInitial = frozenset(self.etatsInitiaux),
            estFinal = lambda e: not self.etatsFinaux.isdisjoint(e),
            delta = lambda e,x: frozenset(self.delta1(e,x))
        )

class AFD_fonctionnel:
    """Automate fini deterministe"""

    def __init__(self, symboles, etatInitial, estFinal, delta):
        self.symboles = symboles
        self.etatInitial = etatInitial
        self.estFinal  = estFinal
        self.delta = delta

    def motDeLongueurMinimale(self):
        q = deque([([],self.etatInitial)])
        cont = True
        etats = {self.etatInitial}
        while q:
            mot, etat = q.pop()
            if etat is not None: # etat n'est pas un état puit
                if self.estFinal(etat):
                    return mot
                for symbole in self.symboles:
                    etat_ = self.delta(etat,symbole)
                    if etat_ not in etats:
                        q.appendleft((mot + [symbole],etat_))
                        etats.add(etat_)
        
    def afd(self):
        alphabet = list(self.symboles)
        p = len(alphabet)
        t = dict()          # {etat : numero,..}
        f = deque()         # file des etats restant a traiter
        n = 0               # nombre d'etats
        etatsFinaux = set()
        delta = [] 
        def numeroOfEtat(q):
            # l'etat q est rajoute a t et a f si necessaire
            nonlocal n
            try:
                return t[q]
            except KeyError :
                t[q] = n
                f.append(q)
                if self.estFinal(q):
                    etatsFinaux.add(n)
                n += 1
                return n - 1
        _ = numeroOfEtat(self.etatInitial)
        while f:
            q = f.popleft()
            l = []
            for i in range(p):
                l.append(numeroOfEtat(self.delta(q,alphabet[i])))
            delta.append(l)
        return AFD(alphabet, etatsFinaux, delta)
    
    def complementaire(self):
        return AFD_fonctionnel(self.symboles, self.etatInitial, (lambda q: not self.estFinal(q)), self.delta)

 
class AFD:
    """Automate fini deterministe"""    

    def __init__(self, symboles, etatsFinaux, delta):
        self.symboles = symboles
        self.etatsFinaux  = etatsFinaux
        self.delta = delta

    def __str__(self,largeur = 3):
        p = len(self.symboles)
        n = len(self.delta)
        motif = '{0:^' + str(largeur) + '}'
        s = 'Etat initial : 0\n'
        s += 'Etats finaux : {}\n'.format(self.etatsFinaux)
        s += (' ' * largeur) + '|'
        for x in self.symboles:
            s += motif.format(x)
        s += '\n' + '-' * largeur + '+' + '-' * (largeur * p) + '\n'
        for q in range(n):
            s += motif.format(q) + '|'
            for i in range(p):
                s += motif.format(self.delta[q][i])
            s += '\n'
        return s        
        
    def motsDeLongueurMinimale(self,long = -1):
        """si long >= 0, renvoie les mots de longueur long"""
        if self.etatsFinaux:
            p = len(self.symboles)
            n = len(self.delta)
            l = []
            for q in range(n):
                l.append({''} if q in self.etatsFinaux else set())
            t = [l]
            k = 0
            while k < long or (long == -1 and t[-1][0] == set()):
                k += 1
                l = []
                for q in range(n):
                    e = set()
                    for x in range(p):
                        for v in t[-1][self.delta[q][x]]:
                            e.add(self.symboles[x] + v)
                    l.append(e)
                t.append(l)
            return t[-1][0]
        else:
            return set()

### Test

In [2]:
# mots se terminant par 'aba'
af = AF(
    symboles = {'a','b'},
    etats = {1,2,3,4},
    etatsInitiaux = {1},
    etatsFinaux = {4},
    transitions = {
        1 : {'a' : {1,2}, 'b' : {1}},
        2 : {'b' : {3}},
        3 : {'a' : {4}}
    }
)

afd_fonctionnel = af.afd_fonctionnel()
print('Un plus petit mot du langage : {}\n'.format(afd_fonctionnel.motDeLongueurMinimale()))

afd = afd_fonctionnel.afd()  
print(afd)
print('Tous les plus petits mots du langage : {}'.format(afd.motsDeLongueurMinimale()))

Un plus petit mot du langage : ['a', 'b', 'a']

Etat initial : 0
Etats finaux : {3}
   | a  b 
---+------
 0 | 1  0 
 1 | 1  2 
 2 | 3  0 
 3 | 1  2 

Tous les plus petits mots du langage : {'aba'}


## 2. Le problème des 3 bouteilles

3 bouteilles de contenances $u$, $v$ et $w$ litres contiennent initialement $a$, $b$ et $c$ litres.

Le but est d'obtenir $x$ litres dans l'une des bouteilles en versant les bouteilles les unes dans les autres successivement.

In [3]:
def troisBouteilles(contenances, contenus, x):
    
    # (i,j) = vider (une part de) la bouteille i dans j
    symboles = {(i,j) for i in range(3) for j in range(3) if i != j}
    
    etatInitial = contenus
    
    def estFinal(q):
        return q[0] == x or q[1] == x or q[2] == x
    
    def delta(q,s):
        i,j = s
        l = list(q)
        if l[i] and l[j] < contenances[j]:
            if l[i] + l[j] <= contenances[j]:
                l[j] += l[i]
                l[i] = 0
            else:
                l[i] -= contenances[j] - l[j]
                l[j] = contenances[j]
            return tuple(l)
        
    a = AFD_fonctionnel(symboles, etatInitial, estFinal, delta)
    sol = a.motDeLongueurMinimale()
    
    def str_etat(q):
        a, b, c = q
        return f'{a:>2} {b:>2} {c:>2}'
    
    q = etatInitial    
    print(f'\n{"contenances":<19}{str_etat(contenances)}')
    print(f'{"état initial":<19}{str_etat(q)}')
    for i,j in sol:
        q = delta(q,(i,j))
        print(f'verser {i+1} dans {j+1} -> {str_etat(q)}')
        
troisBouteilles((8,5,3),(8,0,0),4)
troisBouteilles((10,5,6),(10,1,0),8)


contenances         8  5  3
état initial        8  0  0
verser 1 dans 2 ->  3  5  0
verser 2 dans 3 ->  3  2  3
verser 3 dans 1 ->  6  2  0
verser 2 dans 3 ->  6  0  2
verser 1 dans 2 ->  1  5  2
verser 2 dans 3 ->  1  4  3

contenances        10  5  6
état initial       10  1  0
verser 1 dans 3 ->  4  1  6
verser 3 dans 2 ->  4  5  2
verser 2 dans 1 ->  9  0  2
verser 3 dans 2 ->  9  2  0
verser 1 dans 3 ->  3  2  6
verser 3 dans 2 ->  3  5  3
verser 2 dans 1 ->  8  0  3


## 3. [Loup chèvre chou](https://fr.wikipedia.org/wiki/Le_loup,_la_ch%C3%A8vre_et_le_chou)

Sur une rive d'un fleuve, il y a 3 humains, 1 grand singe, 2 petits singes
et 1 barque.  
Les humains et le grand singe sont capables de manœuvrer la barque,
pas les petits singes.  
La barque peut convenir à 1 ou 2 occupants.  
De plus, il ne faut pas que les humains soient en infériorité numérique,
sinon ils se font attaquer par les singes.  
Comment emmener tout ce petit monde de l’autre côté du fleuve ?

In [4]:
# Un état est un quadruplet
# nombre d'humains, de grands singes, de petits singes, de barques
# sur la rive gauche (rive où se trouve tout le monde au début)

symboles = {('H','H'), ('H','S'), ('H','s'), ('H',), ('S','s'), ('S',)}

etatInitial = 3,1,2,1

def estFinal(q):
    return q == (0,0,0,0)

def estCorrect(q):
    H,S,s,_ = q
    return H >= 0 and S >= 0 and s >= 0 and ( H == 0 or H >= S + s)

def etatOppose(q):
    H,S,s,B = q
    return 3 - H, 1 - S, 2 - s, 1 - B

def delta(q, l):
    H, S, s, B = q
    eps = -1 if B else 1
    for x in l:
        if x == 'H':
            H += eps
        elif x == 'S':
            S += eps
        elif x == 's':
            s += eps 
    p = H, S, s, 1 - B
    if estCorrect(p) and estCorrect(etatOppose(p)):
        return H, S, s, 1 - B
    
automate = AFD_fonctionnel(symboles, etatInitial, estFinal, delta)

sol = automate.motDeLongueurMinimale()
   
def str_etat(q):
    def str_elt(e,n,m):
        return f'{e:>2}' * n + ' ' * 2 * (m - n)
    def aux(q):
        H, S, s, B = q
        return str_elt('H',H,3) + str_elt('S',S,1) + str_elt('s',s,2) + str_elt('B',B,1) + ' ' * 10
    return aux(q) + aux(etatOppose(q))
    
q = etatInitial
rg, rd = 'rive gauche', 'rive droite'
print(' ' * 4 + f'{rg:<24}' + f'{rd:<24}traversée\n')
for i,l in enumerate(sol):
    fl1,fl2 =  (' <- ', '   ') if i % 2 else ('    ', ' ->')
    print(f'{i:>2} ' + str_etat(q),end = '')
    print(fl1 + l[0],end = ' ')
    print((l[1] if len(l) == 2 else ' ') + fl2)
    q = automate.delta(q,l)
print(f'{len(sol):>2} ' + str_etat(q)) 

    rive gauche             rive droite             traversée

 0  H H H S s s B                                      H s ->
 1  H H   S s               H       s   B           <- H     
 2  H H H S s   B                   s                  S s ->
 3  H H H                         S s s B           <- S     
 4  H H H S     B                   s s                H H ->
 5  H     S                 H H     s s B           <- H s   
 6  H H   S s   B           H       s                  H S ->
 7  H       s               H H   S s   B           <- H s   
 8  H H     s s B           H     S                    H H ->
 9          s s             H H H S     B           <- S     
10        S s s B           H H H                      S s ->
11          s               H H H S s   B           <- H     
12  H       s   B           H H   S s                  H s ->
13                          H H H S s s B          


## 4. Jeu sur un graphe

Les lettres $A,B,C,D,E$  ont été mélangées. À chaque coup,
une lettre va de sa zone à la zone vide, à condition qu'il
existe un chemin y menant.
Il s'agit de replacer  les lettres comme indiqué :  
![graphe](graphe.jpg)  
Il y a une solution simple en 12 coups : $DEABCDEABCDE$.

Trouver une solution en 10 coups.

In [7]:
graphe =[[1,5],[0,2,3,4],[1,3,4,5],[1,2,4,5],[1,2,3,5],[0,2,3,4]]

symboles = {'A', 'B', 'C', 'D', 'E'}

etatInitial = 'vDEABC'

def estFinal(q):
    return q == 'vABCDE'

def delta(q,x):
    if q is None: return None
    i, j = q.index(x), q.index('v')
    if j in graphe[i]:
        def f(k): return x if k == j else 'v' if k == i else q[k]
        return f(0)+f(1)+f(2)+f(3)+f(4)+f(5)

automate = AFD_fonctionnel(symboles, etatInitial, estFinal, delta).afd()
    
l = automate.motsDeLongueurMinimale(long = 10)

print(len(l),'solutions',end = ' ')
for s in l: print(s, end = ' ')

9 solutions DEACDEABDE CAEBDCAECA CBDEBCEACE DACDABDEBA CABDCAEBCA DECDEACBDE CADCAEBDCA DEBACDEADE DEBCDEACDE 

## 5. Le problème des 4 jetons (d'après [Jean-Eric Pin](https://www.irif.fr/~jep/indexAnglais.html)) 
### Enoncé

Le joueur a les yeux bandés. Face à lui, un plateau sur lequel sont disposés en carré quatre jetons, blancs d'un côté et noirs de l'autre. Le but du jeu est d'avoir les quatre jetons du côté blanc. Pour cela, le joueur peut retourner autant de jetons qu'il le souhaite, mais sans les déplacer. A chaque tour, le maître de jeu annonce si la configuration obtenue est gagnante ou pas, puis effectue une rotation du plateau de zéro, un, deux ou trois quarts de tours. La configuration de départ est inconnue du joueur, mais le maître de jeu annonce avant le début du jeu qu'elle n'est pas gagnante. Chaque annonce prend une seconde, et il faut 3 secondes au joueur pour retourner les jetons. Pouvez-vous aider le joueur à gagner en moins d'une minute ?

![fig](JEPin.jpg)

### Solution

Le plateau peut avoir six configurations différentes :
- *0* : les 4 jetons sont du côté blanc
- *1* : 1 jeton est noir
- *a* : 2 jetons sont blancs et ils sont adjacents
- *o* : 2 jetons sont blancs et ils sont diagonalement opposés
- *3* : 1 jeton est blanc
- *4* : les 4 jetons sont noirs

Seule la configuration *0* est gagnante.

Le joueur peut retourner un jeton, deux jetons adjacents, deux jetons opposés, trois jetons ou quatre jetons. Nous noterons respectivement *1*, *a*, *o*, *3* et *4* ces différentes actions. Les transitions définies par ces actions sont représentées par l'automate `af` suivant dans lequel toutes les transitions arrivant sur la configuration gagnante ont été omises. Comme tous les états sont finaux, cet automate reconnait le complémentaire du langage cherché.

In [6]:
X = {'1','a','o','3','4'}

af = AF(
    symboles = X,
    etats  = X,
    etatsInitiaux = X,
    etatsFinaux = X,
    transitions = {
      '1' : {'1' : {'a','o'},     'a' : {'1','3'}, 'o' : {'1','3'}, '3' : {'a','o','4'}, '4' : {'3'}},
      'a' : {'1' : {'1','3'},     'a' : {'o','4'}, 'o' : {'a'},     '3' : {'1','3'},     '4' : {'a'}},
      'o' : {'1' : {'1','3'},     'a' : {'a'},     'o' : {'4'},     '3' : {'1','3'},     '4' : {'o'}},
      '3' : {'1' : {'a','o','4'}, 'a' : {'1','3'}, 'o' : {'1','3'}, '3' : {'a','o'},     '4' : {'1'}},
      '4' : {'1' : {'3'},         'a' : {'a'},     'o' : {'o'},     '3' : {'1'}}
    }
)

afd = af.afd_fonctionnel().complementaire()

for x in afd.motDeLongueurMinimale(): print(x, end = '')

4o4a4o434o4a4o4